Create the following table and then analisis it

| Tag | nbo. projects | nbo. lenders | nbo. loans | total amount | community yes/no? |
|-----|---------------|--------------|------------|---------------|-------------------|
|vegan ||||||


In [ ]:
import pandas as pd
import cudf
from tqdm import tqdm
from matplotlib import pyplot as plt

tqdm.pandas()

In [ ]:
vn = cudf.read_parquet("checkpoints/vn_since_20200101.parquet")
vn["sector_name"] = vn["sector_name"].astype("category")
vn["geocode_country_name"] = vn["geocode_country_name"].astype("category")
vn["activity_name"] = vn["activity_name"].astype("category")
vn["tags"] = vn["tags"].astype("category")
vn.info()

In [ ]:
vn.to_csv("checkpoints/vn_since_20200101.csv")

In [ ]:
vn.rename(columns={"id": "project_id"}, inplace=True)
vn.head(10)

some basic stats, just to check again

In [ ]:
vn.duplicated().sum()

In [ ]:
vn.project_id.nunique()

In [ ]:
vn.lender_id.nunique()

In [ ]:
# The number of Loans
len(vn.drop(columns=["tags"]).drop_duplicates())

create a column call `relative_shareAmount`, which is the 

$$\text{relative\_shareAmount} = \frac{shareAmount}{loanAmount} \times 100$$

In [ ]:
vn["relative_shareAmount"] = vn["shareAmount"] / vn["loanAmount"] * 100

since each line as a unique loan action, I will create a column call `loan_id` that corresponding to each line

In [ ]:
import cupy as cp

vn["loan_id"] = cp.arange(len(vn))  # not true at all

In [ ]:
taggr = vn.groupby("tags")

In [ ]:
nb_projects = taggr.project_id.nunique().rename("nb_projects")
nb_projects.head(2)

In [ ]:
nb_lender = taggr.lender_id.nunique().rename("nb_lender")
nb_lender.head(2)

In [ ]:
nb_loan = taggr.loan_id.nunique().rename("nb_loan")
nb_loan.head(2)

In [ ]:
project_tag_df = vn[["project_id", "loanAmount", "tags"]].drop_duplicates()
project_tag_df.head(3)

In [ ]:
total_amount = project_tag_df.groupby("tags").loanAmount.sum().rename("total_amount")
total_amount.head(2)

In [ ]:
table = cudf.concat([nb_projects, nb_lender, nb_loan, total_amount], axis=1)
table = table.sort_values(by="nb_projects", ascending=False)
table

# Metrics

Define metric1 as

$$\text{metric1} = \frac{\text{nb\_loan}}{\text{nb\_lenders}}$$

- if this metric is large, indiciate that on average, people willing to give money multiple time rather then when the metric is small
- if this metric is large, indicating that it might be a community involved

In [ ]:
total_number_of_loan = len(vn.drop(columns=["tags"]).drop_duplicates())
total_number_of_lender = vn.lender_id.nunique()
total_number_of_loan / total_number_of_lender

In [ ]:
total_number_of_lender

In [ ]:
table["metric1"] = table["nb_loan"] / table["nb_lender"]
table["avg_money_per_project"] = table["total_amount"] / table["nb_projects"]
table

metric2. The ideal is if a loans (a donation) contain more money, it indicate that the funder very interested in the project and willing to contribute more money  
If the number of loans (donations) for a project is small, it indicates that people willing to give large amount of money to that project.

In [ ]:
plt.figure(figsize=(20, 20))
for i, (name, group) in enumerate(taggr):
    plt.subplot(5, 5, i + 1)
    name = vn["tags"].cat.categories[name]
    # plt.figure() # Creates a new figure
    cpu_group = group.to_pandas()
    cpu_group["relative_shareAmount"].plot(
        kind="hist", bins=20, title=name, ylabel=None
    )  # Replace 'your_column' with the column you want to plot
    plt.xlabel(None)
    plt.ylabel(None)
plt.show()